In [ ]:
## results are in the subfolder named "saved" of the drive folder :- https://drive.google.com/drive/folders/1_Nt3xnZJpVlI0jRJQM4QJ4vdMKjSSmZn?usp=sharing 
## detailed result analysis :- https://docs.google.com/document/d/1nRHeqZVKCQ7sC9tGEDbVhdMrRtnR3Gv627Lfv6HEB2s/edit?usp=sharing 
## output of taining GAN is after this cell :- https://colab.research.google.com/drive/1yIEu_yICdh2B3MiXK4nyXu07V_PRdgmG#scrollTo=rhoyxiOYe0Mm&line=1&uniqifier=1

In [ ]:
from google.colab import drive
drive.mount('/content/My_drive')

Mounted at /content/My_drive


In [ ]:
import os
import torch
from copy import deepcopy
from random import shuffle
import torch.nn.functional as F

def d_hinge_loss(real_pred, fake_pred):
    real_loss = F.relu(1-real_pred)
    fake_loss = F.relu(1+fake_pred)

    return real_loss.mean() + fake_loss.mean()


def g_hinge_loss(pred):
    return -pred.mean()


class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def true_randperm(size, device='cuda'):
    def unmatched_randperm(size):
        l1 = [i for i in range(size)]
        l2 = []
        for j in range(size):
            deleted = False
            if j in l1:
                deleted = True
                del l1[l1.index(j)]
            shuffle(l1)
            if len(l1) == 0:
                return 0, False
            l2.append(l1[0])
            del l1[0]
            if deleted:
                l1.append(j)
        return l2, True
    flag = False
    l = torch.zeros(size).long()
    while not flag:
        l, flag = unmatched_randperm(size)
    return torch.LongTensor(l).to(device)


def copy_G_params(model):
    flatten = deepcopy(list(p.data for p in model.parameters()))
    return flatten


def load_params(model, new_param):
    for p, new_p in zip(model.parameters(), new_param):
        p.data.copy_(new_p)


def make_folders(save_folder, trial_name):
    saved_model_folder = os.path.join(save_folder, 'train_results/%s/models'%trial_name)
    saved_image_folder = os.path.join(save_folder, 'train_results/%s/images'%trial_name)
    folders = [os.path.join(save_folder, 'train_results'), 
               os.path.join(save_folder, 'train_results/%s'%trial_name), 
               os.path.join(save_folder, 'train_results/%s/images'%trial_name), 
               os.path.join(save_folder, 'train_results/%s/models'%trial_name)]
    for folder in folders:
        if not os.path.exists(folder):
            os.mkdir(folder)
    
    from shutil import copy
    try:
        for f in os.listdir('.'):
            if '.py' in f:
                copy(f, os.path.join(save_folder, 'train_results/%s'%trial_name)+'/'+f)
    except:
        pass
    return saved_image_folder, saved_model_folder 



import cv2
import numpy as np
import math


def warp(img, mag=10, freq=100):
    rows, cols = img.shape

    img_output = np.zeros(img.shape, dtype=img.dtype)

    for i in range(rows):
        for j in range(cols):
            offset_x = int(mag * math.sin(2 * 3.14 * i / freq))
            offset_y = int(mag * math.cos(2 * 3.14 * j / freq))
            if i+offset_y < rows and j+offset_x < cols:
                img_output[i,j] = img[(i+offset_y)%rows,(j+offset_x)%cols]
            else:
                img_output[i,j] = 0

    return img_output


In [ ]:
import datetime

DATA_NAME = 'art'

DATALOADER_WORKERS = 4
NBR_CLS = 25

EPOCH_GAN = 10

SAVE_IMAGE_INTERVAL = 10
SAVE_MODEL_INTERVAL = 10
LOG_INTERVAL = 10
FID_INTERVAL = 10
FID_BATCH_NBR = 10

ITERATION_AE = 250

NFC=32
MULTI_GPU = False


IM_SIZE_GAN = 512
BATCH_SIZE_GAN = 4

IM_SIZE_AE = 512
BATCH_SIZE_AE = 8

ct = datetime.datetime.now()  
TRIAL_NAME = 'trial-pr-%s-%d-%d-%d-%d'%(DATA_NAME, ct.month, ct.day, ct.hour, ct.minute)
SAVE_FOLDER = '/content/My_drive/MyDrive/rajat-Inspiron-3576/SEM7/DEEP_LEARNING/Project/Files/saved'

PRETRAINED_AE_PATH = None
PRETRAINED_AE_ITER = 12000

GAN_CKECKPOINT =None

TRAIN_AE_ONLY = False
TRAIN_GAN_ONLY = False

data_root_colorful = '/content/My_drive/MyDrive/rajat-Inspiron-3576/SEM7/DEEP_LEARNING/Project/Files/images/rgb_images'
data_root_sketch_1 = '/content/My_drive/MyDrive/rajat-Inspiron-3576/SEM7/DEEP_LEARNING/Project/Files/sketch/sketch-1'
data_root_sketch_2 = '/content/My_drive/MyDrive/rajat-Inspiron-3576/SEM7/DEEP_LEARNING/Project/Files/sketch/sketch-2'
data_root_sketch_3 = '/content/My_drive/MyDrive/rajat-Inspiron-3576/SEM7/DEEP_LEARNING/Project/Files/saved/train_results/synthesized_sketches'

In [ ]:
import os
import random
import numpy as np
from PIL import Image, ImageFilter
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torchvision.transforms.functional as Fff
import torch.utils.data as data

normalize_imagenet = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

def _noise_adder(img):
    return torch.empty_like(img, dtype=img.dtype).uniform_(0.0, 1/128.0) + img


def _rescale(img):
    return img * 2.0 - 1.0


def trans_maker(size=512):
    trans = transforms.Compose([
                    transforms.Resize((size, size)),
                    transforms.ToTensor(),
                    _rescale
                    ])
    return trans


def trans_maker_augment(size=256):
    trans = transforms.Compose([ 
                    transforms.Resize((int(size*1.1),int(size*1.1))),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomCrop((size, size)),
                    transforms.ToTensor(),
                    _rescale
                    ])
    return trans

class SelfSupervisedDataset(Dataset):
    def __init__(self, data_root, data_root_2, im_size=512, nbr_cls=2000, rand_crop=True):
        super(SelfSupervisedDataset, self).__init__()
        self.root = data_root
        self.skt_root = data_root_2

        self.frame = self._parse_frame()
        random.shuffle(self.frame)

        self.nbr_cls = nbr_cls
        self.set_offset = 0

        self.im_size = im_size
        self.transform_rd = transforms.Compose([ 
                            transforms.Resize((int(im_size*1.3), int(im_size*1.3))),
                            transforms.RandomCrop( (int(im_size), int(im_size)) ),
                            transforms.RandomRotation( 30 ),
                            transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                            _rescale])

        self.crop = rand_crop
        if self.crop:
            self.transform_1 = transforms.Resize((int(im_size*1.1), int(im_size*1.1)))
            self.transform_2 = transforms.Compose([ transforms.ToTensor(),
                                                    _rescale
                                                    ])
            self.rand_range = int(self.im_size * 0.1)
        else:
            self.transform_normal = trans_maker(size=im_size)
    
        self.transform_flip = transforms.RandomHorizontalFlip(p=1)

        self.transform_erase = transforms.Compose([
                        transforms.RandomErasing(p=0.8, scale=(0.02, 0.1), value=1),
                        transforms.RandomErasing(p=0.8, scale=(0.02, 0.1), value=1),
                                ])

    def _parse_frame(self):
        frame = []
        img_names = os.listdir(self.root)
        img_names.sort()
        for i in range(len(img_names)):
            img_name = '%d.jpg'%(i)
            img_name = img_names[i]
            image_path = os.path.join(self.root, img_names[i])
            skt_path = os.path.join( self.skt_root,  img_name)
            if os.path.exists(image_path) and os.path.exists(skt_path): 
                frame.append( (image_path, skt_path) )
        return frame

    def __len__(self):
        return self.nbr_cls

    def _next_set(self):
        self.set_offset += self.nbr_cls
        if self.set_offset > ( len(self.frame) - self.nbr_cls ):
            random.shuffle(self.frame)
            self.set_offset = 0

    def __getitem__(self, idx):
        file, skt_path = self.frame[idx+self.set_offset]
        img = Image.open(file).convert('RGB')
        skt = Image.open(skt_path).convert('L')
        
        bold_factor = 3 
        skt_bold = skt.filter( ImageFilter.MinFilter(size=bold_factor) )

        if random.randint(0, 1) == 1:
            img = self.transform_flip(img)
            skt = self.transform_flip(skt)
            skt_bold = self.transform_flip(skt_bold)

        img_rd = self.transform_rd(img) 

        if self.crop:
            img_normal = self.transform_1(img) 
            skt_normal = self.transform_1(skt) 
            skt_bold = self.transform_1(skt_bold) 

            i = random.randint(0, self.rand_range) 
            j = random.randint(0, self.rand_range) 

            img_normal = Fff.crop(img_normal, i, j, self.im_size, self.im_size)
            skt_normal = Fff.crop(skt_normal, i, j, self.im_size, self.im_size)
            skt_bold = Fff.crop(skt_bold, i, j, self.im_size, self.im_size)

            img_normal = self.transform_2(img_normal) 
            skt_normal = self.transform_2(skt_normal) 
            skt_bold = self.transform_2(skt_bold) 
        else:
            img_normal = self.transform_normal(img)
            skt_normal = self.transform_normal(skt)
            skt_bold = self.transform_normal(skt_bold)

        skt_erased = self.transform_erase(skt_normal)
        skt_erased_bold = self.transform_erase(skt_bold)
        return img_rd, img_normal, skt_normal, skt_bold, skt_erased, skt_erased_bold, idx


class PairedMultiDataset(Dataset):
    def __init__(self, data_root_1, data_root_2, data_root_3, data_root_4, rand_crop=True, im_size=512):
        super(PairedMultiDataset, self).__init__()
        self.root_a = data_root_1
        self.root_b = data_root_2
        self.root_c = data_root_3
        self.root_d = data_root_4

        self.frame = self._parse_frame()
        
        self.crop = rand_crop
        self.im_size = im_size
        if self.crop:
            self.transform_1 = transforms.Resize((int(im_size*1.1), int(im_size*1.1)))
            self.transform_2 = transforms.Compose([ transforms.ToTensor(),
                                                    _rescale
                                                    ])
            self.rand_range = int(self.im_size * 0.1)
        else:
            self.transform = trans_maker( int( im_size ) )

    def _parse_frame(self):
        frame = []

        img_names = os.listdir(self.root_a)
        img_names.sort()
        for i in range(len(img_names)):
            img_name = '%d.jpg'%(i)
            img_name = img_names[i]
            image_a_path = os.path.join(self.root_a, img_names[i])
            if os.path.exists(image_a_path): 
                image_b_path = os.path.join(self.root_b, img_name)
                image_c_path = os.path.join(self.root_c, img_name)
                image_d_path = os.path.join(self.root_d, img_name)
                if os.path.exists(image_b_path) and os.path.exists(image_c_path) and os.path.exists(image_d_path):
                    frame.append( (image_a_path, image_b_path, image_c_path, image_d_path) )
                else:
                    print('2', image_a_path, image_b_path)
            else:
                print("1", image_a_path)
        return frame

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        file_a, file_b, file_c, file_d = self.frame[idx]
        img_a = Image.open(file_a).convert('RGB')
        img_b = Image.open(file_b).convert('L')
        img_c = Image.open(file_c).convert('L')
        img_d = Image.open(file_d).convert('L')
            
        if self.crop:
            img_a = self.transform_1(img_a) 
            img_b = self.transform_1(img_b) 
            img_c = self.transform_1(img_c) 
            img_d = self.transform_1(img_d) 

            i = random.randint(0, self.rand_range) 
            j = random.randint(0, self.rand_range) 
            img_a = Fff.crop(img_a, i, j, self.im_size, self.im_size)
            img_b = Fff.crop(img_b, i, j, self.im_size, self.im_size)
            img_c = Fff.crop(img_c, i, j, self.im_size, self.im_size)
            img_d = Fff.crop(img_d, i, j, self.im_size, self.im_size)

            img_a = self.transform_2(img_a) 
            img_b = self.transform_2(img_b) 
            img_c = self.transform_2(img_c) 
            img_d = self.transform_2(img_d) 
        else:
            img_a = self.transform(img_a) 
            img_b = self.transform(img_b) 
            img_c = self.transform(img_c) 
            img_d = self.transform(img_d) 


        return (img_a, img_b, img_c, img_d)


class PairedDataset(Dataset):
    def __init__(self, data_root_1, data_root_2, transform=trans_maker(512)):
        super(PairedDataset, self).__init__()
        self.root_a = data_root_1
        self.root_b = data_root_2

        self.frame = self._parse_frame()
        self.transform = transform


    def _parse_frame(self):
        frame = []
        img_names = os.listdir(self.root_a)
        img_names.sort()
        for i in range(len(img_names)):
            img_name = '%d.jpg'%(i)
            if DATA_NAME == 'shoe':
                img_name = img_names[i]
            image_a_path = os.path.join(self.root_a, img_names[i])
            if ('.jpg' in image_a_path) or ('.png' in image_a_path): 
                image_b_path = os.path.join(self.root_b, img_name)
                if os.path.exists(image_b_path):
                    frame.append( (image_a_path, image_b_path) )

        return frame

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        file_a, file_b = self.frame[idx]
        img_a = Image.open(file_a).convert('RGB')
        img_b = Image.open(file_b).convert('L')
            
        if self.transform:
            img_a = self.transform(img_a) 
            img_b = self.transform(img_b) 

        return (img_a, img_b)


class  ImageFolder(Dataset):
    def __init__(self, data_root, transform=trans_maker(512)):
        super( ImageFolder, self).__init__()
        self.root = data_root

        self.frame = self._parse_frame()
        self.transform = transform

    def _parse_frame(self):
        frame = []
        img_names = os.listdir(self.root)
        img_names.sort()
        for i in range(len(img_names)):
            image_path = os.path.join(self.root, img_names[i])
            if ('.jpg' in image_path) or ('.png' in image_path): 
                frame.append(image_path)

        return frame

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        file = self.frame[idx]
        img = Image.open(file).convert('RGB')
            
        if self.transform:
            img = self.transform(img) 
        return img




def InfiniteSampler(n):
    i = n - 1
    order = np.random.permutation(n)
    while True:
        yield order[i]
        i += 1
        if i >= n:
            np.random.seed()
            order = np.random.permutation(n)
            i = 0


class InfiniteSamplerWrapper(data.sampler.Sampler):
    def __init__(self, data_source):
        self.num_samples = len(data_source)

    def __iter__(self):
        return iter(InfiniteSampler(self.num_samples))

    def __len__(self):
        return 2 ** 31

In [ ]:
!pip install torch_dwconv

     |████████████████████████████████| 249 kB 5.3 MB/s 
  Created wheel for torch-dwconv: filename=torch_dwconv-0.1.0-cp37-cp37m-linux_x86_64.whl size=2644617 sha256=82ccfbc69b1857f813c5a6a7973ea6788339d59ed73097703b43dacb51f37d34
  Stored in directory: /root/.cache/pip/wheels/fc/ce/e6/33361c4cae3eb5d8649e1fce38efdb2f1013f01fa586b81f31
Successfully built torch-dwconv


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import spectral_norm
from torch_dwconv import DepthwiseConv2d

import math
import random


def weights_init(m):
    classname = m.__class__.__name__
    try:
        if classname.find('Conv') != -1:
            m.weight.data.normal_(0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            m.weight.data.normal_(1.0, 0.02)
            m.bias.data.fill_(0)
    except:
        pass


class DMI(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        
        self.weight_a = nn.Parameter(torch.ones(1, in_channels, 1, 1)*1.01)
        self.weight_b = nn.Parameter(torch.ones(1, in_channels, 1, 1)*0.99)

        self.bias_a = nn.Parameter(torch.zeros(1, in_channels, 1, 1)+0.01)
        self.bias_b = nn.Parameter(torch.zeros(1, in_channels, 1, 1)-0.01)

    def forward(self, feat, mask):
        if feat.shape[1] > mask.shape[1]:
            channel_scale = feat.shape[1] // mask.shape[1]
            mask = mask.repeat(1, channel_scale, 1, 1)
        
        mask = F.interpolate(mask, size=feat.shape[2])
        feat_a = self.weight_a * feat * mask + self.bias_a
        feat_b = self.weight_b * feat * (1-mask) + self.bias_b
        return feat_a + feat_b


class Swish(nn.Module):
    def forward(self, feat):
        return feat * torch.sigmoid(feat)


class Squeeze(nn.Module):
    def forward(self, feat):
        return feat.squeeze(-1).squeeze(-1)


class UnSqueeze(nn.Module):
    def forward(self, feat):
        return feat.unsqueeze(-1).unsqueeze(-1)


class GLU(nn.Module):
    def __init__(self):
        super(GLU, self).__init__()

    def forward(self, x):
        nc = x.size(1)
        assert nc % 2 == 0, 'channels dont divide 2!'
        nc = int(nc/2)
        return x[:, :nc] * torch.sigmoid(x[:, nc:])


class NoiseInjection(nn.Module):
    def __init__(self, ch):
        super().__init__()

        self.weight = nn.Parameter(torch.zeros(1, ch, 1, 1), requires_grad=True)

    def forward(self, feat, noise=None):
        if noise is None:
            batch, _, height, width = feat.shape
            noise = torch.randn(batch, 1, height, width).to(feat.device)

        return feat + self.weight * noise


class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            spectral_norm( nn.Linear(channel, channel // reduction, bias=False) ),
            nn.ReLU(inplace=True),
            spectral_norm( nn.Linear(channel // reduction, channel, bias=False) ),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


class ResBlkG(nn.Module):
    def __init__(self, ch, ch_m=4):
        super().__init__()
        self.main = nn.Sequential(spectral_norm( nn.BatchNorm2d(ch) ),
                            spectral_norm( nn.Conv2d(ch, ch*ch_m, 1, 1, 0, bias=False) ),
                            spectral_norm( nn.BatchNorm2d(ch*ch_m) ), Swish(),
                            spectral_norm( DepthwiseConv2d(ch*ch_m, ch*ch_m, 5, 1, 2) ),
                            spectral_norm( nn.BatchNorm2d(ch*ch_m) ), Swish(),
                            spectral_norm( nn.Conv2d(ch*ch_m, ch, 1, 1, 0, bias=False) ),
                            spectral_norm( nn.BatchNorm2d(ch) ),
                            SELayer(ch))
    def forward(self, feat):
        return feat + self.main(feat)


class ResBlkE(nn.Module):
    def __init__(self, ch):
        super().__init__()

        self.main = nn.Sequential(
                            spectral_norm( nn.BatchNorm2d(ch) ), Swish(),
                            spectral_norm( nn.Conv2d(ch, ch, 3, 1, 1, bias=False) ),
                            spectral_norm( nn.BatchNorm2d(ch) ), Swish(),
                            spectral_norm( nn.Conv2d(ch, ch, 3, 1, 1, bias=False) ),
                            SELayer(ch))

    def forward(self, feat):
        return feat + self.main(feat)


class StyleEncoder(nn.Module):
    def __init__(self, nfc=64, nbr_cls=500):
        super().__init__()

        self.nfc = nfc

        self.sf_256 = nn.Sequential(nn.Conv2d(3, nfc//4, 4, 2, 1, bias=False),nn.LeakyReLU(0.2,inplace=True))
        self.sf_128 = nn.Sequential(nn.Conv2d(nfc//4, nfc//2, 4, 2, 1, bias=False),nn.BatchNorm2d(nfc//2),nn.LeakyReLU(0.1,inplace=True)) 
        self.sf_64 = nn.Sequential(nn.Conv2d(nfc//2, nfc, 4, 2, 1, bias=False),nn.BatchNorm2d(nfc),nn.LeakyReLU(0.1,inplace=True)) 
        
        self.sf_32 = nn.Sequential(nn.Conv2d(nfc, nfc*2, 4, 2, 1, bias=False), ResBlkE(nfc*2))
        self.sf_16 = nn.Sequential(nn.LeakyReLU(0.1,inplace=True), nn.Conv2d(nfc*2, nfc*4, 4, 2, 1, bias=False), ResBlkE(nfc*4))
        self.sf_8 = nn.Sequential(nn.LeakyReLU(0.1,inplace=True), nn.Conv2d(nfc*4, nfc*8, 4, 2, 1, bias=False), ResBlkE(nfc*8))
        
        self.sfv_32 = nn.Sequential( nn.AdaptiveAvgPool2d(output_size=4), nn.Conv2d(nfc*2, nfc*2, 4, 1, 0, bias=False), Squeeze() )
        self.sfv_16 = nn.Sequential( nn.AdaptiveAvgPool2d(output_size=4), nn.Conv2d(nfc*4, nfc*4, 4, 1, 0, bias=False), Squeeze() )
        self.sfv_8 = nn.Sequential( nn.AdaptiveAvgPool2d(output_size=4), nn.Conv2d(nfc*8, nfc*8, 4, 1, 0, bias=False), Squeeze() )

        self.nbr_cls = nbr_cls
        self.final_cls = None

    def reset_cls(self):
        if self.final_cls is None:
            self.final_cls = nn.Sequential(nn.LeakyReLU(0.1), nn.Linear(self.nfc*8, self.nbr_cls))
        stdv = 1. / math.sqrt(self.final_cls[1].weight.size(1))
        self.final_cls[1].weight.data.uniform_(-stdv, stdv)
        if self.final_cls[1].bias is not None:
            self.final_cls[1].bias.data.uniform_(-0.1*stdv, 0.1*stdv)

    def get_feats(self, image):
        feat = self.sf_256(image)
        feat = self.sf_128(feat)
        feat = self.sf_64(feat)
        feat_32 = self.sf_32(feat)
        feat_16 = self.sf_16(feat_32)
        feat_8 = self.sf_8(feat_16)
        
        feat_32 = self.sfv_32(feat_32)
        feat_16 = self.sfv_16(feat_16)
        feat_8 = self.sfv_8(feat_8)

        return feat_32, feat_16, feat_8

    def forward(self, image):
        feat_32, feat_16, feat_8 = self.get_feats(image)

        pred_cls = self.final_cls(feat_8)
        return [feat_32, feat_16, feat_8], pred_cls


class ContentEncoder(nn.Module):
    def __init__(self, nfc=64):
        super().__init__()

        self.cf_256 = nn.Sequential(nn.Conv2d(1, nfc//4, 4, 2, 1, bias=False),nn.LeakyReLU(0.2,inplace=True))
        self.cf_128 = nn.Sequential(nn.Conv2d(nfc//4, nfc//2, 4, 2, 1, bias=False),nn.BatchNorm2d( nfc//2),nn.LeakyReLU(0.1,inplace=True)) 
        self.cf_64 = nn.Sequential(nn.Conv2d( nfc//2, nfc, 4, 2, 1, bias=False),nn.BatchNorm2d(nfc),nn.LeakyReLU(0.1,inplace=True)) 
        
        self.cf_32 = nn.Sequential(nn.Conv2d(nfc, nfc*2, 4, 2, 1, bias=False), ResBlkE(nfc*2))
        self.cf_16 = nn.Sequential(nn.LeakyReLU(0.1,inplace=True), nn.Conv2d(nfc*2, nfc*4, 4, 2, 1, bias=False), ResBlkE(nfc*4))
        self.cf_8 = nn.Sequential(nn.LeakyReLU(0.1,inplace=True), nn.Conv2d(nfc*4, nfc*8, 4, 2, 1, bias=False), ResBlkE(nfc*8))
        
    def get_feats(self, image):
        feat = self.cf_256(image)
        feat = self.cf_128(feat)
        feat = self.cf_64(feat)
        feat_32 = self.cf_32(feat)
        feat_16 = self.cf_16(feat_32)
        feat_8 = self.cf_8(feat_16)

        return feat_32, feat_16, feat_8

    def forward(self, image):
        feat_32, feat_16, feat_8 = self.get_feats(image)
        return [feat_32, feat_16, feat_8]


def up_decoder(ch_in, ch_out):
    return nn.Sequential(
        nn.UpsamplingNearest2d(scale_factor=2),
        nn.Conv2d(ch_in, ch_out*2, 3, 1, 1, bias=False),
        nn.InstanceNorm2d( ch_out*2 ), GLU())


class Decoder(nn.Module):
    def __init__(self, nfc=64):
        super().__init__()
 
        self.base_feat = nn.Parameter(torch.randn(1, nfc*8, 8, 8).normal_(0, 1), requires_grad=True)
        
        self.dmi_8 = DMI(nfc*8)
        self.dmi_16 = DMI(nfc*4)

        self.feat_8_1 = nn.Sequential( ResBlkG(nfc*16), nn.LeakyReLU(0.1,inplace=True), nn.Conv2d(nfc*16, nfc*8, 3, 1, 1, bias=False), nn.InstanceNorm2d(nfc*8) )
        self.feat_8_2 = nn.Sequential( nn.LeakyReLU(0.1,inplace=True), ResBlkG(nfc*8) )
        self.feat_16  = nn.Sequential( nn.LeakyReLU(0.1,inplace=True), nn.UpsamplingNearest2d(scale_factor=2), nn.Conv2d(nfc*8, nfc*4, 3, 1, 1, bias=False), ResBlkG(nfc*4) )
        self.feat_32  = nn.Sequential( nn.LeakyReLU(0.1,inplace=True), nn.UpsamplingNearest2d(scale_factor=2), nn.Conv2d(nfc*8, nfc*2, 3, 1, 1, bias=False), ResBlkG(nfc*2) )
        self.feat_64  = nn.Sequential( nn.LeakyReLU(0.1,inplace=True), up_decoder(nfc*4, nfc) ) 
        self.feat_128 = up_decoder(nfc*1, nfc//2)
        self.feat_256 = up_decoder(nfc//2, nfc//4)
        self.feat_512 = up_decoder(nfc//4, nfc//8)
        
        self.to_rgb = nn.Sequential( nn.Conv2d(nfc//8, 3, 3, 1, 1, bias=False), nn.Tanh() )
        
        self.style_8 = nn.Sequential( nn.Linear(nfc*8, nfc*8), nn.ReLU(), nn.Linear(nfc*8, nfc*8), nn.BatchNorm1d(nfc*8), UnSqueeze() )
        self.style_64 = nn.Sequential( nn.Linear(nfc*8, nfc), nn.ReLU(), nn.Linear(nfc, nfc), nn.Sigmoid() , UnSqueeze())
        self.style_128 = nn.Sequential( nn.Linear(nfc*4, nfc//2), nn.ReLU(), nn.Linear(nfc//2, nfc//2), nn.Sigmoid() , UnSqueeze())
        self.style_256 = nn.Sequential( nn.Linear(nfc*2, nfc//4), nn.ReLU(), nn.Linear(nfc//4, nfc//4), nn.Sigmoid() , UnSqueeze())

    def forward(self, content_feats, style_vectors):

        feat_8 = self.feat_8_1( torch.cat( [content_feats[2], self.base_feat.repeat(style_vectors[0].shape[0], 1, 1, 1)], dim=1 ) )            
        feat_8 = self.dmi_8(feat_8, content_feats[2])

        bs = feat_8.shape[0]

        feat_8 = feat_8 * self.style_8( style_vectors[2] )
        feat_8 = self.feat_8_2(feat_8)

        feat_16 = self.feat_16(feat_8) 
        feat_16 = self.dmi_16(feat_16, content_feats[1])
        feat_16 = torch.cat([feat_16, content_feats[1]], dim=1)

        feat_32 = self.feat_32(feat_16) 
        feat_32 = torch.cat([feat_32, content_feats[0]], dim=1)

        feat_64 = self.feat_64(feat_32) * self.style_64(style_vectors[2]) 
        feat_128 = self.feat_128(feat_64) * self.style_128(style_vectors[1]) 
        feat_256 = self.feat_256(feat_128) * self.style_256(style_vectors[0]) 
        feat_512 = self.feat_512(feat_256) 

        return self.to_rgb(feat_512)


class AE(nn.Module):
    def __init__(self, nfc, nbr_cls=500):
        super().__init__()  

        self.style_encoder = StyleEncoder(nfc, nbr_cls=nbr_cls)
        self.content_encoder = ContentEncoder(nfc)
        self.decoder = Decoder(nfc)

    @torch.no_grad()
    def forward(self, skt_img, style_img):
        style_feats = self.style_encoder.get_feats( F.interpolate(style_img, size=512) )
        content_feats = self.content_encoder( F.interpolate( skt_img , size=512) )
        gimg = self.decoder(content_feats, style_feats)
        return gimg, style_feats

    def load_state_dicts(self, path):
        ckpt = torch.load(path)
        self.style_encoder.reset_cls()
        self.style_encoder.load_state_dict(ckpt['s'])
        self.content_encoder.load_state_dict(ckpt['c'])
        self.decoder.load_state_dict(ckpt['d'])
        print('AE load success')

def down_gan(ch_in, ch_out):
    return nn.Sequential(
        spectral_norm(nn.Conv2d(ch_in, ch_out, 4, 2, 1, bias=False)),
        nn.BatchNorm2d(ch_out), nn.LeakyReLU(0.2, inplace=True))


def up_gan(ch_in, ch_out):
    return nn.Sequential(
        nn.UpsamplingNearest2d(scale_factor=2),
        spectral_norm( nn.Conv2d(ch_in, ch_out*2, 3, 1, 1, bias=False) ),
        nn.BatchNorm2d( ch_out*2 ), NoiseInjection(ch_out*2), GLU())


def repeat_upscale(feat, scale_factor=2):
    feat = feat.repeat(1,1,scale_factor,scale_factor)
    return feat


class RefineGenerator_art(nn.Module):
    def __init__(self, nfc=64, im_size=512):
        super().__init__()  

        self.im_size = im_size

        d16, d32, d64, d128, d256, d512 = nfc*8, nfc*8, nfc*4, nfc*2, nfc, nfc//2 

        self.from_noise_32 = nn.Sequential( UnSqueeze(),
            spectral_norm(nn.ConvTranspose2d(nfc*8, nfc*8, 4, 1, 0, bias=False)), #4
            nn.BatchNorm2d(nfc*8), GLU(), up_gan(nfc*4, nfc*2),  up_gan(nfc*2, nfc*2), up_gan(nfc*2, nfc*1)) #32 

        self.from_style = nn.Sequential( UnSqueeze(),
            spectral_norm(nn.ConvTranspose2d(nfc*(8+4+2), nfc*16, 4, 1, 0, bias=False)), #4
            nn.BatchNorm2d(nfc*16), GLU(), up_gan(nfc*8, nfc*4) )
        
        self.encode_256 = nn.Sequential( spectral_norm(nn.Conv2d(3, d256, 4, 2, 1, bias=False)),nn.LeakyReLU(0.2,inplace=True))
        self.encode_128 = down_gan(d256, d128)
        self.encode_64 = down_gan(d128, d64)
        self.encode_32 = down_gan(d64, d32)
        self.encode_16 = down_gan(d32, d16)

        self.residual_16 = nn.Sequential( ResBlkG(d16+nfc*4), Swish(), ResBlkG(d16+nfc*4), Swish() )

        self.decode_32  = up_gan(d16+nfc*4, d32)
        self.decode_64  = up_gan(d32+nfc, d64) 
        self.decode_128 = up_gan(d64, d128)
        self.decode_256 = up_gan(d128, d256)
        self.decode_512 = up_gan(d256, d512)
        if im_size == 1024:
            self.decode_1024 = up_gan(d512, nfc//4)

        self.style_64  =  nn.Sequential( spectral_norm( nn.Linear(nfc*8, d64) ), nn.ReLU(), nn.Linear(d64, d64),  nn.Sigmoid(), UnSqueeze())
        self.style_128 =  nn.Sequential( spectral_norm( nn.Linear(nfc*8, d128)), nn.ReLU(), nn.Linear(d128, d128),nn.Sigmoid(), UnSqueeze())
        self.style_256 =  nn.Sequential( spectral_norm( nn.Linear(nfc*4, d256)), nn.ReLU(), nn.Linear(d256, d256),nn.Sigmoid(), UnSqueeze())
        self.style_512 =  nn.Sequential( spectral_norm( nn.Linear(nfc*2, d512)), nn.ReLU(), nn.Linear(d512, d512),nn.Sigmoid(), UnSqueeze())
        
        self.to_rgb = nn.Sequential( nn.Conv2d(nfc//2, 3, 3, 1, 1, bias=False), nn.Tanh() )
        if im_size == 1024:
            self.to_rgb = nn.Sequential( nn.Conv2d(nfc//4, 3, 3, 1, 1, bias=False), nn.Tanh() )
        
        if DATA_NAME=='shoe':
            self.bs_0 = nn.Parameter(torch.randn(1, nfc*2))
            self.bs_1 = nn.Parameter(torch.randn(1, nfc*4))
            self.bs_2 = nn.Parameter(torch.randn(1, nfc*8))

    def forward(self, image, style_vectors):
         
        s_16 = repeat_upscale( self.from_style(torch.cat(style_vectors,1)), scale_factor=2 )
        if DATA_NAME=='shoe':  
            s_16 = torch.zeros_like(s_16)
            
        n_32 = self.from_noise_32(torch.randn_like(style_vectors[2]))

        e_256 = self.encode_256( image )
        e_128 = self.encode_128( e_256 )
        e_64 = self.encode_64( e_128 )
        e_32 = self.encode_32( e_64 )
        e_16 = self.encode_16(e_32)

        e_16 = self.residual_16( torch.cat([e_16, s_16],dim=1) )
        
        d_32 = self.decode_32( e_16 )
        d_64 = self.decode_64( torch.cat([d_32, n_32], dim=1) ) 
        if DATA_NAME!='shoe':
            d_64 *= self.style_64(style_vectors[2])
        d_128 = self.decode_128( d_64 + e_64 ) 
        if DATA_NAME!='shoe':
            d_128 *= self.style_128(style_vectors[2])
        d_256 = self.decode_256( d_128 + e_128 )
        if DATA_NAME!='shoe':
            d_256 *= self.style_256(style_vectors[1])
        d_512 = self.decode_512( d_256 + e_256 ) 
        if DATA_NAME!='shoe':
            d_512 *= self.style_512(style_vectors[0])
        
        if self.im_size == 1024:
            d_final = self.decode_1024(d_512)
        else:
            d_final = d_512
        return self.to_rgb(d_final)


class RefineGenerator_face(nn.Module):
    def __init__(self, nfc, im_size):
        super().__init__()  

        self.im_size = im_size

        e1, e2, e3, e4 = 16, 32, 64, 128
        self.encode_1 = down_gan(3, e1)      
        self.encode_2 = down_gan(e1, e2)     
        self.encode_3 = down_gan(e2, e3)     
        self.encode_4 = down_gan(e3, e4)    

        s1, s2, s3, s4 = 256, 128, 128, 64
        self.style = nn.Sequential(nn.Linear(nfc*(8+4+2), 512), nn.LeakyReLU(0.1))
        self.from_style_32 = nn.Sequential(
            spectral_norm(nn.ConvTranspose2d(512, s1, 4, 1, 0, bias=False)), 
            nn.BatchNorm2d(s1), GLU(), up_gan(s1//2, s2), up_gan(s2, s3), up_gan(s3, s4)) 

        d1, d2, d3, d4, d5 = 256, 128, 64, 32, 16
        self.decode_64 = up_gan( e4 + s4 , d1)
        self.decode_128 = up_gan(d1+e3, d2)
        self.decode_256 = up_gan(d2+e2, d3)
        self.decode_512 = up_gan(d3+e1, d4)
        if im_size == 1024:
            self.decode_1024 = up_gan(d4, d5)

        self.style_blocks = nn.ModuleList()
        
        chs = [d1, d2, d3, d4]
        if im_size == 1024:
            chs.append(d5)
        for i in range(len(chs)):
            self.style_blocks.append(nn.Sequential( 
                    nn.Linear(512, chs[i]), nn.ReLU(), nn.Linear(chs[i], chs[i]), nn.Sigmoid() ))

        self.final = nn.Sequential( spectral_norm( 
                            nn.Conv2d(d4, 3, 3, 1, 1, bias=False) ), nn.Tanh() )
        if im_size == 1024:
            self.final = nn.Sequential( spectral_norm( 
                            nn.Conv2d(d5, 3, 3, 1, 1, bias=False) ), nn.Tanh() )
        
    def forward(self, image, style):
        e_256 = self.encode_1( image )
        e_128 = self.encode_2( e_256 )
        e_64 = self.encode_3( e_128 )
        e_32 = self.encode_4( e_64 )

        style = self.style(torch.cat(style, dim=1))
        s_32 = self.from_style_32( style.unsqueeze(-1).unsqueeze(-1) )
        
        if random.randint(0, 1) == 1:
            s_32 = s_32.flip(2)
        if random.randint(0, 1) == 1:
            s_32 = s_32.flip(3)
        
        feat_64 = self.decode_64( torch.cat([e_32, s_32], dim=1) ) * self.style_blocks[0](style).unsqueeze(-1).unsqueeze(-1)
        feat_128 = self.decode_128( torch.cat([e_64, feat_64], dim=1) ) * self.style_blocks[1](style).unsqueeze(-1).unsqueeze(-1)
        feat_256 = self.decode_256( torch.cat([e_128, feat_128], dim=1) ) * self.style_blocks[2](style).unsqueeze(-1).unsqueeze(-1)
        feat_512 = self.decode_512( torch.cat([e_256, feat_256], dim=1) ) * self.style_blocks[3](style).unsqueeze(-1).unsqueeze(-1)
        if self.im_size == 1024:
            feat_1024 = self.decode_1024( feat_512 ) * self.style_blocks[4](style).unsqueeze(-1).unsqueeze(-1)
            return self.final(feat_1024)
        else:
            return self.final(feat_512)


class DownBlock(nn.Module):
    def __init__(self, ch_in, ch_out, ch_skip=0):
        super().__init__()

        self.ch_out = ch_out
        self.down_main = nn.Sequential(
                spectral_norm(nn.Conv2d(ch_in, ch_out, 3, 2, 1, bias=False)),
                nn.BatchNorm2d(ch_out),
                nn.LeakyReLU(0.2, inplace=True),
                spectral_norm(nn.Conv2d(ch_out, ch_out, 3, 1, 1, bias=False)),
                nn.BatchNorm2d(ch_out),
                nn.LeakyReLU(0.2, inplace=True)
                )

        self.skip = False
        
        if ch_skip > 0:  
            self.skip = True 
            self.skip_conv = nn.Sequential(
                nn.AdaptiveAvgPool2d(4),
                spectral_norm( nn.Conv2d(ch_skip, ch_out, 4, 1, 0, bias=False) ),
                nn.ReLU(),
                spectral_norm( nn.Conv2d(ch_out, ch_out*2, 1, 1, 0, bias=False) ),
            ) 

    def forward(self, feat, skip_feat=None):
        feat_out = self.down_main(feat) 
        if skip_feat is not None and self.skip:
            addon = self.skip_conv(skip_feat)
            feat_out = feat_out * torch.sigmoid(addon[:,:self.ch_out]) + torch.tanh(addon[:,self.ch_out:])        
        
        return feat_out


class Discriminator(nn.Module):
    def __init__(self, ndf=64, nc=3, im_size=512):
        super(Discriminator, self).__init__()
        self.ndf = ndf
        self.im_size = im_size

        modules = [
            nn.Sequential(spectral_norm(nn.Conv2d(nc, ndf//4, 4, 2, 1, bias=False)),
                          nn.LeakyReLU(0.2, inplace=True)),
            DownBlock(ndf//4, ndf//2),
            DownBlock(ndf//2, ndf*1),
            DownBlock(ndf*1,  ndf*2),
            DownBlock(ndf*2,  ndf*4, ch_skip=ndf//4),
            ]

        if im_size == 512:
            modules.append(
                DownBlock(ndf*4,  ndf*16, ch_skip=ndf//2),
            )
        elif im_size == 1024:
            modules.append(
                DownBlock(ndf*4,  ndf*8, ch_skip=ndf//2))
            modules.append(
                DownBlock(ndf*8,  ndf*16, ch_skip=ndf*1),
            )
        modules.append(
                        nn.Sequential(
                            spectral_norm(nn.Conv2d(ndf*16, ndf*16, 1, 1, 0, bias=False)),
                            nn.BatchNorm2d(ndf*16),
                            nn.LeakyReLU(0.2, inplace=True),
                            spectral_norm(nn.Conv2d(ndf*16, 1, 4, 1, 0, bias=False)))
                       )

        self.main = nn.ModuleList(modules)
        
        self.apply(weights_init)


    def forward(self, x):
        feat_256 = self.main[0](x)
        feat_128 = self.main[1](feat_256)
        feat_64 = self.main[2](feat_128)
        feat_32 = self.main[3](feat_64)

        feat_16 = self.main[4](feat_32, feat_256)
        feat_8 = self.main[5](feat_16, feat_128)
        if self.im_size == 1024:
            feat_last = self.main[6](feat_8, feat_64)
        else:
            feat_last = feat_8

        return self.main[-1](feat_last)

In [ ]:
import os
import torch
from torch.autograd import Variable
from pdb import set_trace as st
from IPython import embed

class BaseModel():
    def __init__(self):
        pass;
        
    def name(self):
        return 'BaseModel'

    def initialize(self, use_gpu=True, gpu_ids=[0]):
        self.use_gpu = use_gpu
        self.gpu_ids = gpu_ids

    def forward(self):
        pass

    def get_image_paths(self):
        pass

    def optimize_parameters(self):
        pass

    def get_current_visuals(self):
        return self.input

    def get_current_errors(self):
        return {}

    def save(self, label):
        pass

    def save_network(self, network, path, network_label, epoch_label):
        save_filename = '%s_net_%s.pth' % (epoch_label, network_label)
        save_path = os.path.join(path, save_filename)
        torch.save(network.state_dict(), save_path)

    def load_network(self, network, network_label, epoch_label):
        save_filename = '%s_net_%s.pth' % (epoch_label, network_label)
        save_path = os.path.join(self.save_dir, save_filename)
        print('Loading network from %s'%save_path)
        network.load_state_dict(torch.load(save_path))

    def update_learning_rate():
        pass

    def get_image_paths(self):
        return self.image_paths

    def save_done(self, flag=False):
        np.save(os.path.join(self.save_dir, 'done_flag'),flag)
        np.savetxt(os.path.join(self.save_dir, 'done_flag'),[flag,],fmt='%i')

In [ ]:
from collections import namedtuple
import torch
from torchvision import models as tv
from IPython import embed

class squeezenet(torch.nn.Module):
    def __init__(self, requires_grad=False, pretrained=True):
        super(squeezenet, self).__init__()
        pretrained_features = tv.squeezenet1_1(pretrained=pretrained).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        self.slice6 = torch.nn.Sequential()
        self.slice7 = torch.nn.Sequential()
        self.N_slices = 7
        for x in range(2):
            self.slice1.add_module(str(x), pretrained_features[x])
        for x in range(2,5):
            self.slice2.add_module(str(x), pretrained_features[x])
        for x in range(5, 8):
            self.slice3.add_module(str(x), pretrained_features[x])
        for x in range(8, 10):
            self.slice4.add_module(str(x), pretrained_features[x])
        for x in range(10, 11):
            self.slice5.add_module(str(x), pretrained_features[x])
        for x in range(11, 12):
            self.slice6.add_module(str(x), pretrained_features[x])
        for x in range(12, 13):
            self.slice7.add_module(str(x), pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h = self.slice1(X)
        h_relu1 = h
        h = self.slice2(h)
        h_relu2 = h
        h = self.slice3(h)
        h_relu3 = h
        h = self.slice4(h)
        h_relu4 = h
        h = self.slice5(h)
        h_relu5 = h
        h = self.slice6(h)
        h_relu6 = h
        h = self.slice7(h)
        h_relu7 = h
        vgg_outputs = namedtuple("SqueezeOutputs", ['relu1','relu2','relu3','relu4','relu5','relu6','relu7'])
        out = vgg_outputs(h_relu1,h_relu2,h_relu3,h_relu4,h_relu5,h_relu6,h_relu7)

        return out


class alexnet(torch.nn.Module):
    def __init__(self, requires_grad=False, pretrained=True):
        super(alexnet, self).__init__()
        alexnet_pretrained_features = tv.alexnet(pretrained=pretrained).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        self.N_slices = 5
        for x in range(2):
            self.slice1.add_module(str(x), alexnet_pretrained_features[x])
        for x in range(2, 5):
            self.slice2.add_module(str(x), alexnet_pretrained_features[x])
        for x in range(5, 8):
            self.slice3.add_module(str(x), alexnet_pretrained_features[x])
        for x in range(8, 10):
            self.slice4.add_module(str(x), alexnet_pretrained_features[x])
        for x in range(10, 12):
            self.slice5.add_module(str(x), alexnet_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h = self.slice1(X)
        h_relu1 = h
        h = self.slice2(h)
        h_relu2 = h
        h = self.slice3(h)
        h_relu3 = h
        h = self.slice4(h)
        h_relu4 = h
        h = self.slice5(h)
        h_relu5 = h
        alexnet_outputs = namedtuple("AlexnetOutputs", ['relu1', 'relu2', 'relu3', 'relu4', 'relu5'])
        out = alexnet_outputs(h_relu1, h_relu2, h_relu3, h_relu4, h_relu5)

        return out

class vgg16(torch.nn.Module):
    def __init__(self, requires_grad=False, pretrained=True):
        super(vgg16, self).__init__()
        vgg_pretrained_features = tv.vgg16(pretrained=pretrained).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        self.N_slices = 5
        for x in range(4):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(4, 9):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(9, 16):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(16, 23):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        for x in range(23, 30):
            self.slice5.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h = self.slice1(X)
        h_relu1_2 = h
        h = self.slice2(h)
        h_relu2_2 = h
        h = self.slice3(h)
        h_relu3_3 = h
        h = self.slice4(h)
        h_relu4_3 = h
        h = self.slice5(h)
        h_relu5_3 = h
        vgg_outputs = namedtuple("VggOutputs", ['relu1_2', 'relu2_2', 'relu3_3', 'relu4_3', 'relu5_3'])
        out = vgg_outputs(h_relu1_2, h_relu2_2, h_relu3_3, h_relu4_3, h_relu5_3)

        return out



class resnet(torch.nn.Module):
    def __init__(self, requires_grad=False, pretrained=True, num=18):
        super(resnet, self).__init__()
        if(num==18):
            self.net = tv.resnet18(pretrained=pretrained)
        elif(num==34):
            self.net = tv.resnet34(pretrained=pretrained)
        elif(num==50):
            self.net = tv.resnet50(pretrained=pretrained)
        elif(num==101):
            self.net = tv.resnet101(pretrained=pretrained)
        elif(num==152):
            self.net = tv.resnet152(pretrained=pretrained)
        self.N_slices = 5

        self.conv1 = self.net.conv1
        self.bn1 = self.net.bn1
        self.relu = self.net.relu
        self.maxpool = self.net.maxpool
        self.layer1 = self.net.layer1
        self.layer2 = self.net.layer2
        self.layer3 = self.net.layer3
        self.layer4 = self.net.layer4

    def forward(self, X):
        h = self.conv1(X)
        h = self.bn1(h)
        h = self.relu(h)
        h_relu1 = h
        h = self.maxpool(h)
        h = self.layer1(h)
        h_conv2 = h
        h = self.layer2(h)
        h_conv3 = h
        h = self.layer3(h)
        h_conv4 = h
        h = self.layer4(h)
        h_conv5 = h

        outputs = namedtuple("Outputs", ['relu1','conv2','conv3','conv4','conv5'])
        out = outputs(h_relu1, h_conv2, h_conv3, h_conv4, h_conv5)

        return out

In [ ]:
!pip install lpips

     |████████████████████████████████| 53 kB 1.6 MB/s 


In [ ]:
from __future__ import absolute_import
import sys
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
import numpy as np
from pdb import set_trace as st
from skimage import color
from IPython import embed

import lpips as util

def spatial_average(in_tens, keepdim=True):
    return in_tens.mean([2,3],keepdim=keepdim)

def upsample(in_tens, out_H=64):
    in_H = in_tens.shape[2]
    scale_factor = 1.*out_H/in_H

    return nn.Upsample(scale_factor=scale_factor, mode='bilinear', align_corners=False)(in_tens)

class PNetLin(nn.Module):
    def __init__(self, pnet_type='vgg', pnet_rand=False, pnet_tune=False, use_dropout=True, spatial=False, version='0.1', lpips=True):
        super(PNetLin, self).__init__()

        self.pnet_type = pnet_type
        self.pnet_tune = pnet_tune
        self.pnet_rand = pnet_rand
        self.spatial = spatial
        self.lpips = lpips
        self.version = version
        self.scaling_layer = ScalingLayer()

        if(self.pnet_type in ['vgg','vgg16']):
            net_type = vgg16
            self.chns = [64,128,256,512,512]
        elif(self.pnet_type=='alex'):
            net_type = alexnet
            self.chns = [64,192,384,256,256]
        elif(self.pnet_type=='squeeze'):
            net_type = squeezenet
            self.chns = [64,128,256,384,384,512,512]
        self.L = len(self.chns)

        self.net = net_type(pretrained=not self.pnet_rand, requires_grad=self.pnet_tune)

        if(lpips):
            self.lin0 = NetLinLayer(self.chns[0], use_dropout=use_dropout)
            self.lin1 = NetLinLayer(self.chns[1], use_dropout=use_dropout)
            self.lin2 = NetLinLayer(self.chns[2], use_dropout=use_dropout)
            self.lin3 = NetLinLayer(self.chns[3], use_dropout=use_dropout)
            self.lin4 = NetLinLayer(self.chns[4], use_dropout=use_dropout)
            self.lins = [self.lin0,self.lin1,self.lin2,self.lin3,self.lin4]
            if(self.pnet_type=='squeeze'):
                self.lin5 = NetLinLayer(self.chns[5], use_dropout=use_dropout)
                self.lin6 = NetLinLayer(self.chns[6], use_dropout=use_dropout)
                self.lins+=[self.lin5,self.lin6]

    def forward(self, in0, in1, retPerLayer=False):
        in0_input, in1_input = (self.scaling_layer(in0), self.scaling_layer(in1)) if self.version=='0.1' else (in0, in1)
        outs0, outs1 = self.net.forward(in0_input), self.net.forward(in1_input)
        feats0, feats1, diffs = {}, {}, {}

        for kk in range(self.L):
            feats0[kk], feats1[kk] = util.normalize_tensor(outs0[kk]), util.normalize_tensor(outs1[kk])
            diffs[kk] = (feats0[kk]-feats1[kk])**2

        if(self.lpips):
            if(self.spatial):
                res = [upsample(self.lins[kk].model(diffs[kk]), out_H=in0.shape[2]) for kk in range(self.L)]
            else:
                res = [spatial_average(self.lins[kk].model(diffs[kk]), keepdim=True) for kk in range(self.L)]
        else:
            if(self.spatial):
                res = [upsample(diffs[kk].sum(dim=1,keepdim=True), out_H=in0.shape[2]) for kk in range(self.L)]
            else:
                res = [spatial_average(diffs[kk].sum(dim=1,keepdim=True), keepdim=True) for kk in range(self.L)]

        val = res[0]
        for l in range(1,self.L):
            val += res[l]
        
        if(retPerLayer):
            return (val, res)
        else:
            return val

class ScalingLayer(nn.Module):
    def __init__(self):
        super(ScalingLayer, self).__init__()
        self.register_buffer('shift', torch.Tensor([-.030,-.088,-.188])[None,:,None,None])
        self.register_buffer('scale', torch.Tensor([.458,.448,.450])[None,:,None,None])

    def forward(self, inp):
        return (inp - self.shift) / self.scale


class NetLinLayer(nn.Module):
    def __init__(self, chn_in, chn_out=1, use_dropout=False):
        super(NetLinLayer, self).__init__()

        layers = [nn.Dropout(),] if(use_dropout) else []
        layers += [nn.Conv2d(chn_in, chn_out, 1, stride=1, padding=0, bias=False),]
        self.model = nn.Sequential(*layers)


class Dist2LogitLayer(nn.Module):
    def __init__(self, chn_mid=32, use_sigmoid=True):
        super(Dist2LogitLayer, self).__init__()

        layers = [nn.Conv2d(5, chn_mid, 1, stride=1, padding=0, bias=True),]
        layers += [nn.LeakyReLU(0.2,True),]
        layers += [nn.Conv2d(chn_mid, chn_mid, 1, stride=1, padding=0, bias=True),]
        layers += [nn.LeakyReLU(0.2,True),]
        layers += [nn.Conv2d(chn_mid, 1, 1, stride=1, padding=0, bias=True),]
        if(use_sigmoid):
            layers += [nn.Sigmoid(),]
        self.model = nn.Sequential(*layers)

    def forward(self,d0,d1,eps=0.1):
        return self.model.forward(torch.cat((d0,d1,d0-d1,d0/(d1+eps),d1/(d0+eps)),dim=1))

class BCERankingLoss(nn.Module):
    def __init__(self, chn_mid=32):
        super(BCERankingLoss, self).__init__()
        self.net = Dist2LogitLayer(chn_mid=chn_mid)
        self.loss = torch.nn.BCELoss()

    def forward(self, d0, d1, judge):
        per = (judge+1.)/2.
        self.logit = self.net.forward(d0,d1)
        return self.loss(self.logit, per)

class FakeNet(nn.Module):
    def __init__(self, use_gpu=True, colorspace='Lab'):
        super(FakeNet, self).__init__()
        self.use_gpu = use_gpu
        self.colorspace=colorspace

class L2(FakeNet):

    def forward(self, in0, in1, retPerLayer=None):
        assert(in0.size()[0]==1)

        if(self.colorspace=='RGB'):
            (N,C,X,Y) = in0.size()
            value = torch.mean(torch.mean(torch.mean((in0-in1)**2,dim=1).view(N,1,X,Y),dim=2).view(N,1,1,Y),dim=3).view(N)
            return value
        elif(self.colorspace=='Lab'):
            value = util.l2(util.tensor2np(util.tensor2tensorlab(in0.data,to_norm=False)), 
                util.tensor2np(util.tensor2tensorlab(in1.data,to_norm=False)), range=100.).astype('float')
            ret_var = Variable( torch.Tensor((value,) ) )
            if(self.use_gpu):
                ret_var = ret_var.cuda()
            return ret_var

class DSSIM(FakeNet):

    def forward(self, in0, in1, retPerLayer=None):
        assert(in0.size()[0]==1)

        if(self.colorspace=='RGB'):
            value = util.dssim(1.*util.tensor2im(in0.data), 1.*util.tensor2im(in1.data), range=255.).astype('float')
        elif(self.colorspace=='Lab'):
            value = util.dssim(util.tensor2np(util.tensor2tensorlab(in0.data,to_norm=False)), 
                util.tensor2np(util.tensor2tensorlab(in1.data,to_norm=False)), range=100.).astype('float')
        ret_var = Variable( torch.Tensor((value,) ) )
        if(self.use_gpu):
            ret_var = ret_var.cuda()
        return ret_var

def print_network(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    print('Network',net)
    print('Total number of parameters: %d' % num_params)

In [ ]:
from __future__ import absolute_import
import sys
import numpy as np
import torch
from torch import nn
import os
from collections import OrderedDict
from torch.autograd import Variable
import itertools
from scipy.ndimage import zoom
import fractions
import functools
import skimage.transform
from tqdm import tqdm

from IPython import embed
import lpips as util

class DistModel(BaseModel):
    def name(self):
        return self.model_name

    def initialize(self, model='net-lin', net='alex', colorspace='Lab', pnet_rand=False, pnet_tune=False, model_path=None,
            use_gpu=True, printNet=False, spatial=False, 
            is_train=False, lr=.0001, beta1=0.5, version='0.1', gpu_ids=[0]):
        
        BaseModel.initialize(self, use_gpu=use_gpu, gpu_ids=gpu_ids)

        self.model = model
        self.net = net
        self.is_train = is_train
        self.spatial = spatial
        self.gpu_ids = gpu_ids
        self.model_name = '%s [%s]'%(model,net)

        if(self.model == 'net-lin'):
            self.net = PNetLin(pnet_rand=pnet_rand, pnet_tune=pnet_tune, pnet_type=net,
                use_dropout=True, spatial=spatial, version=version, lpips=True)
            kw = {}
            if not use_gpu:
                kw['map_location'] = 'cpu'
            if(model_path is None):
                import inspect
                model_path = os.path.abspath(os.path.join(inspect.getfile(self.initialize), '..', 'weights/v%s/%s.pth'%(version,net)))

            if(not is_train):
                print('Loading model from: %s'%model_path)
                self.net.load_state_dict(torch.load(model_path, **kw), strict=False)

        elif(self.model=='net'):
            self.net = PNetLin(pnet_rand=pnet_rand, pnet_type=net, lpips=False)
        elif(self.model in ['L2','l2']):
            self.net = L2(use_gpu=use_gpu,colorspace=colorspace)
            self.model_name = 'L2'
        elif(self.model in ['DSSIM','dssim','SSIM','ssim']):
            self.net = DSSIM(use_gpu=use_gpu,colorspace=colorspace)
            self.model_name = 'SSIM'
        else:
            raise ValueError("Model [%s] not recognized." % self.model)

        self.parameters = list(self.net.parameters())

        if self.is_train:
            self.rankLoss = BCERankingLoss()
            self.parameters += list(self.rankLoss.net.parameters())
            self.lr = lr
            self.old_lr = lr
            self.optimizer_net = torch.optim.Adam(self.parameters, lr=lr, betas=(beta1, 0.999))
        else:
            self.net.eval()

        if(use_gpu):
            self.net.to(gpu_ids[0])
            self.net = torch.nn.DataParallel(self.net, device_ids=gpu_ids)
            if(self.is_train):
                self.rankLoss = self.rankLoss.to(device=gpu_ids[0])

        if(printNet):
            print('---------- Networks initialized -------------')
            print_network(self.net)
            print('-----------------------------------------------')

    def forward(self, in0, in1, retPerLayer=False):
        
        return self.net.forward(in0, in1, retPerLayer=retPerLayer)

    def optimize_parameters(self):
        self.forward_train()
        self.optimizer_net.zero_grad()
        self.backward_train()
        self.optimizer_net.step()
        self.clamp_weights()

    def clamp_weights(self):
        for module in self.net.modules():
            if(hasattr(module, 'weight') and module.kernel_size==(1,1)):
                module.weight.data = torch.clamp(module.weight.data,min=0)

    def set_input(self, data):
        self.input_ref = data['ref']
        self.input_p0 = data['p0']
        self.input_p1 = data['p1']
        self.input_judge = data['judge']

        if(self.use_gpu):
            self.input_ref = self.input_ref.to(device=self.gpu_ids[0])
            self.input_p0 = self.input_p0.to(device=self.gpu_ids[0])
            self.input_p1 = self.input_p1.to(device=self.gpu_ids[0])
            self.input_judge = self.input_judge.to(device=self.gpu_ids[0])

        self.var_ref = Variable(self.input_ref,requires_grad=True)
        self.var_p0 = Variable(self.input_p0,requires_grad=True)
        self.var_p1 = Variable(self.input_p1,requires_grad=True)

    def forward_train(self):
        self.d0 = self.forward(self.var_ref, self.var_p0)
        self.d1 = self.forward(self.var_ref, self.var_p1)
        self.acc_r = self.compute_accuracy(self.d0,self.d1,self.input_judge)

        self.var_judge = Variable(1.*self.input_judge).view(self.d0.size())

        self.loss_total = self.rankLoss.forward(self.d0, self.d1, self.var_judge*2.-1.)

        return self.loss_total

    def backward_train(self):
        torch.mean(self.loss_total).backward()

    def compute_accuracy(self,d0,d1,judge):
        d1_lt_d0 = (d1<d0).cpu().data.numpy().flatten()
        judge_per = judge.cpu().numpy().flatten()
        return d1_lt_d0*judge_per + (1-d1_lt_d0)*(1-judge_per)

    def get_current_errors(self):
        retDict = OrderedDict([('loss_total', self.loss_total.data.cpu().numpy()),
                            ('acc_r', self.acc_r)])

        for key in retDict.keys():
            retDict[key] = np.mean(retDict[key])

        return retDict

    def get_current_visuals(self):
        zoom_factor = 256/self.var_ref.data.size()[2]

        ref_img = util.tensor2im(self.var_ref.data)
        p0_img = util.tensor2im(self.var_p0.data)
        p1_img = util.tensor2im(self.var_p1.data)

        ref_img_vis = zoom(ref_img,[zoom_factor, zoom_factor, 1],order=0)
        p0_img_vis = zoom(p0_img,[zoom_factor, zoom_factor, 1],order=0)
        p1_img_vis = zoom(p1_img,[zoom_factor, zoom_factor, 1],order=0)

        return OrderedDict([('ref', ref_img_vis),
                            ('p0', p0_img_vis),
                            ('p1', p1_img_vis)])

    def save(self, path, label):
        if(self.use_gpu):
            self.save_network(self.net.module, path, '', label)
        else:
            self.save_network(self.net, path, '', label)
        self.save_network(self.rankLoss.net, path, 'rank', label)

    def update_learning_rate(self,nepoch_decay):
        lrd = self.lr / nepoch_decay
        lr = self.old_lr - lrd

        for param_group in self.optimizer_net.param_groups:
            param_group['lr'] = lr

        print('update lr [%s] decay: %f -> %f' % (type,self.old_lr, lr))
        self.old_lr = lr

def score_2afc_dataset(data_loader, func, name=''):

    d0s = []
    d1s = []
    gts = []

    for data in tqdm(data_loader.load_data(), desc=name):
        d0s+=func(data['ref'],data['p0']).data.cpu().numpy().flatten().tolist()
        d1s+=func(data['ref'],data['p1']).data.cpu().numpy().flatten().tolist()
        gts+=data['judge'].cpu().numpy().flatten().tolist()

    d0s = np.array(d0s)
    d1s = np.array(d1s)
    gts = np.array(gts)
    scores = (d0s<d1s)*(1.-gts) + (d1s<d0s)*gts + (d1s==d0s)*.5

    return(np.mean(scores), dict(d0s=d0s,d1s=d1s,gts=gts,scores=scores))

def score_jnd_dataset(data_loader, func, name=''):

    ds = []
    gts = []

    for data in tqdm(data_loader.load_data(), desc=name):
        ds+=func(data['p0'],data['p1']).data.cpu().numpy().tolist()
        gts+=data['same'].cpu().numpy().flatten().tolist()

    sames = np.array(gts)
    ds = np.array(ds)

    sorted_inds = np.argsort(ds)
    ds_sorted = ds[sorted_inds]
    sames_sorted = sames[sorted_inds]

    TPs = np.cumsum(sames_sorted)
    FPs = np.cumsum(1-sames_sorted)
    FNs = np.sum(sames_sorted)-TPs

    precs = TPs/(TPs+FPs)
    recs = TPs/(TPs+FNs)
    score = util.voc_ap(recs,precs)

    return(score, dict(ds=ds,sames=sames))

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from skimage.measure import compare_ssim
import torch
from torch.autograd import Variable

class PerceptualLoss(torch.nn.Module):
    def __init__(self, model='net-lin', net='alex', colorspace='rgb', spatial=False, use_gpu=True, gpu_ids=[0]):
        super(PerceptualLoss, self).__init__()
        print('Setting up Perceptual loss...')
        self.use_gpu = use_gpu
        self.spatial = spatial
        self.gpu_ids = gpu_ids
        self.model = DistModel()
        self.model.initialize(model=model, net=net, use_gpu=use_gpu, colorspace=colorspace, spatial=self.spatial, gpu_ids=gpu_ids)
        print('...[%s] initialized'%self.model.name())
        print('...Done')

    def forward(self, pred, target, normalize=False):

        if normalize:
            target = 2 * target  - 1
            pred = 2 * pred  - 1

        return self.model.forward(target, pred)

def normalize_tensor(in_feat,eps=1e-10):
    norm_factor = torch.sqrt(torch.sum(in_feat**2,dim=1,keepdim=True))
    return in_feat/(norm_factor+eps)

def l2(p0, p1, range=255.):
    return .5*np.mean((p0 / range - p1 / range)**2)

def psnr(p0, p1, peak=255.):
    return 10*np.log10(peak**2/np.mean((1.*p0-1.*p1)**2))

def dssim(p0, p1, range=255.):
    return (1 - compare_ssim(p0, p1, data_range=range, multichannel=True)) / 2.

def rgb2lab(in_img,mean_cent=False):
    from skimage import color
    img_lab = color.rgb2lab(in_img)
    if(mean_cent):
        img_lab[:,:,0] = img_lab[:,:,0]-50
    return img_lab

def tensor2np(tensor_obj):
    return tensor_obj[0].cpu().float().numpy().transpose((1,2,0))

def np2tensor(np_obj):
    return torch.Tensor(np_obj[:, :, :, np.newaxis].transpose((3, 2, 0, 1)))

def tensor2tensorlab(image_tensor,to_norm=True,mc_only=False):
    from skimage import color

    img = tensor2im(image_tensor)
    img_lab = color.rgb2lab(img)
    if(mc_only):
        img_lab[:,:,0] = img_lab[:,:,0]-50
    if(to_norm and not mc_only):
        img_lab[:,:,0] = img_lab[:,:,0]-50
        img_lab = img_lab/100.

    return np2tensor(img_lab)

def tensorlab2tensor(lab_tensor,return_inbnd=False):
    from skimage import color
    import warnings
    warnings.filterwarnings("ignore")

    lab = tensor2np(lab_tensor)*100.
    lab[:,:,0] = lab[:,:,0]+50

    rgb_back = 255.*np.clip(color.lab2rgb(lab.astype('float')),0,1)
    if(return_inbnd):
        lab_back = color.rgb2lab(rgb_back.astype('uint8'))
        mask = 1.*np.isclose(lab_back,lab,atol=2.)
        mask = np2tensor(np.prod(mask,axis=2)[:,:,np.newaxis])
        return (im2tensor(rgb_back),mask)
    else:
        return im2tensor(rgb_back)

def rgb2lab(input):
    from skimage import color
    return color.rgb2lab(input / 255.)

def tensor2im(image_tensor, imtype=np.uint8, cent=1., factor=255./2.):
    image_numpy = image_tensor[0].cpu().float().numpy()
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + cent) * factor
    return image_numpy.astype(imtype)

def im2tensor(image, imtype=np.uint8, cent=1., factor=255./2.):
    return torch.Tensor((image / factor - cent)
                        [:, :, :, np.newaxis].transpose((3, 2, 0, 1)))

def tensor2vec(vector_tensor):
    return vector_tensor.data.cpu().numpy()[:, :, 0, 0]

def voc_ap(rec, prec, use_07_metric=False):
    if use_07_metric:
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        
        i = np.where(mrec[1:] != mrec[:-1])[0]
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

def tensor2im(image_tensor, imtype=np.uint8, cent=1., factor=255./2.):
    image_numpy = image_tensor[0].cpu().float().numpy()
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + cent) * factor
    return image_numpy.astype(imtype)

def im2tensor(image, imtype=np.uint8, cent=1., factor=255./2.):
    return torch.Tensor((image / factor - cent)
                        [:, :, :, np.newaxis].transpose((3, 2, 0, 1)))

In [ ]:
import torch
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision import utils as vutils

import random
from tqdm import tqdm

def loss_for_list(loss, fl1, fl2, detach_second=True):
    result_loss = 0
    for f_idx in range(len(fl1)):
        if detach_second:
            result_loss += loss( fl1[f_idx] , fl2[f_idx].detach() )
        else:
            result_loss += loss( fl1[f_idx] , fl2[f_idx] )
    return result_loss


def loss_for_list_perm(loss, fl1, fl2, detach_second=True):
    result_loss = 0
    for f_idx in range(len(fl1)):
        perm = true_randperm(fl1[0].shape[0], fl1[0].device)
        if detach_second:
            result_loss += F.relu( 2 + loss( fl1[f_idx] , fl2[f_idx].detach() ) - loss( fl1[f_idx][perm] , fl2[f_idx].detach() ))
        else:
            result_loss += F.relu( 2 + loss( fl1[f_idx] , fl2[f_idx] ) - loss( fl1[f_idx][perm] , fl2[f_idx] ))
    return result_loss


def loss_for_list_mean(feat_list):
    loss = 0
    for feat in feat_list:
        if len(feat.shape) == 4:
            feat = feat.mean(dim=[2,3])
            loss += F.l1_loss( feat, torch.ones_like(feat) )
        else:
            loss += F.l1_loss( feat, torch.zeros_like(feat) )
    return loss


In [ ]:
!ls

My_drive  sample_data


In [ ]:
!mkdir weights

In [ ]:
!mkdir weights/v0.0

In [ ]:
!mkdir weights/v0.1

In [ ]:
!python -V

Python 3.7.12


In [ ]:
print(torch.__version__)

1.9.0+cu111


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import utils as vutils
from torchvision.datasets import ImageFolder

def make_matrix(dataset_rgb, dataset_skt, net_ae, net_ig, BATCH_SIZE, im_name):
    dataloader_rgb = iter(DataLoader(dataset_rgb, BATCH_SIZE, shuffle=True))
    dataloader_skt = iter(DataLoader(dataset_skt, BATCH_SIZE, shuffle=True))

    rgb_img = next(dataloader_rgb)
    skt_img = next(dataloader_skt)
    
    image_matrix = [ torch.ones(1, 3, 512, 512) ]
    image_matrix.append(rgb_img) 
    with torch.no_grad():
        for skt in skt_img:
            input_skts = skt
            
            gimg_ae, style_feats = net_ae(input_skts, rgb_img)
            g_images = net_ig(gimg_ae, style_feats).cpu()

            image_matrix.append(skt.unsqueeze(0).repeat(1,3,1,1).clone())
            image_matrix.append(gimg_ae.cpu())

            image_matrix.append(skt.unsqueeze(0).repeat(1,3,1,1).clone().fill_(1))
            image_matrix.append( torch.nn.functional.interpolate( g_images , 512 ) )

    image_matrix = torch.cat(image_matrix)
    vutils.save_image(0.5*(image_matrix+1), im_name, nrow=BATCH_SIZE+1)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torchvision.models import inception_v3, Inception3
from torchvision.utils import save_image
from torchvision import utils as vutils
from torch.utils.data import DataLoader

try:
    from torchvision.models.utils import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

import numpy as np
from scipy import linalg
from tqdm import tqdm
import pickle
import os
FID_WEIGHTS_URL = 'https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth'


class InceptionV3(nn.Module):
    DEFAULT_BLOCK_INDEX = 3
    BLOCK_INDEX_BY_DIM = {
        64: 0,   
        192: 1,  
        768: 2,  
        2048: 3 
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False,
                 use_fid_inception=True):
        
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        if use_fid_inception:
            inception = fid_inception_v3()
        else:
            inception = models.inception_v3(pretrained=True)

        
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        
        outp = []
        x = inp

        if self.resize_input:
            x = F.interpolate(x,
                              size=(299, 299),
                              mode='bilinear',
                              align_corners=False)

        if self.normalize_input:
            x = 2 * x - 1  

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp


def fid_inception_v3():
    inception = models.inception_v3(num_classes=1008,
                                    aux_logits=False,
                                    pretrained=False)
    inception.Mixed_5b = FIDInceptionA(192, pool_features=32)
    inception.Mixed_5c = FIDInceptionA(256, pool_features=64)
    inception.Mixed_5d = FIDInceptionA(288, pool_features=64)
    inception.Mixed_6b = FIDInceptionC(768, channels_7x7=128)
    inception.Mixed_6c = FIDInceptionC(768, channels_7x7=160)
    inception.Mixed_6d = FIDInceptionC(768, channels_7x7=160)
    inception.Mixed_6e = FIDInceptionC(768, channels_7x7=192)
    inception.Mixed_7b = FIDInceptionE_1(1280)
    inception.Mixed_7c = FIDInceptionE_2(2048)

    state_dict = load_state_dict_from_url(FID_WEIGHTS_URL, progress=True)
    inception.load_state_dict(state_dict)
    return inception


class FIDInceptionA(models.inception.InceptionA):
    def __init__(self, in_channels, pool_features):
        super(FIDInceptionA, self).__init__(in_channels, pool_features)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1,
                                   count_include_pad=False)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)


class FIDInceptionC(models.inception.InceptionC):
    def __init__(self, in_channels, channels_7x7):
        super(FIDInceptionC, self).__init__(in_channels, channels_7x7)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch7x7 = self.branch7x7_1(x)
        branch7x7 = self.branch7x7_2(branch7x7)
        branch7x7 = self.branch7x7_3(branch7x7)

        branch7x7dbl = self.branch7x7dbl_1(x)
        branch7x7dbl = self.branch7x7dbl_2(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_3(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_4(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_5(branch7x7dbl)

        
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1,
                                   count_include_pad=False)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch7x7, branch7x7dbl, branch_pool]
        return torch.cat(outputs, 1)


class FIDInceptionE_1(models.inception.InceptionE):
    def __init__(self, in_channels):
        super(FIDInceptionE_1, self).__init__(in_channels)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = [
            self.branch3x3_2a(branch3x3),
            self.branch3x3_2b(branch3x3),
        ]
        branch3x3 = torch.cat(branch3x3, 1)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = [
            self.branch3x3dbl_3a(branch3x3dbl),
            self.branch3x3dbl_3b(branch3x3dbl),
        ]
        branch3x3dbl = torch.cat(branch3x3dbl, 1)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1,
                                   count_include_pad=False)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)


class FIDInceptionE_2(models.inception.InceptionE):
    def __init__(self, in_channels):
        super(FIDInceptionE_2, self).__init__(in_channels)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = [
            self.branch3x3_2a(branch3x3),
            self.branch3x3_2b(branch3x3),
        ]
        branch3x3 = torch.cat(branch3x3, 1)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = [
            self.branch3x3dbl_3a(branch3x3dbl),
            self.branch3x3dbl_3b(branch3x3dbl),
        ]
        branch3x3dbl = torch.cat(branch3x3dbl, 1)

        branch_pool = F.max_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)


class Inception3Feature(Inception3):
    def forward(self, x):
        if x.shape[2] != 299 or x.shape[3] != 299:
            x = F.interpolate(x, size=(299, 299), mode='bilinear', align_corners=True)

        x = self.Conv2d_1a_3x3(x)  
        x = self.Conv2d_2a_3x3(x)  
        x = self.Conv2d_2b_3x3(x)  
        x = F.max_pool2d(x, kernel_size=3, stride=2)  

        x = self.Conv2d_3b_1x1(x)  
        x = self.Conv2d_4a_3x3(x)  
        x = F.max_pool2d(x, kernel_size=3, stride=2)

        x = self.Mixed_5b(x)  
        x = self.Mixed_5c(x)  
        x = self.Mixed_5d(x)  

        x = self.Mixed_6a(x)  
        x = self.Mixed_6b(x)  
        x = self.Mixed_6c(x)  
        x = self.Mixed_6d(x)  
        x = self.Mixed_6e(x)  

        x = self.Mixed_7a(x)  
        x = self.Mixed_7b(x)  
        x = self.Mixed_7c(x)  

        x = F.avg_pool2d(x, kernel_size=8)  

        return x.view(x.shape[0], x.shape[1])  


def load_patched_inception_v3():
    inception_feat = InceptionV3([3], normalize_input=False)

    return inception_feat


@torch.no_grad()
def extract_features(loader, inception, device):
    pbar = tqdm(loader)

    feature_list = []

    for img in pbar:
        img = img.to(device)
        feature = inception(img)[0].view(img.shape[0], -1)
        feature_list.append(feature.to('cpu'))

    features = torch.cat(feature_list, 0)

    return features



@torch.no_grad()
def extract_feature_from_samples(generator, inception, device='cuda'):
    n_batch = n_sample // batch_size
    resid = n_sample - (n_batch * batch_size)
    batch_sizes = [batch_size] * n_batch + [resid]
    features = []

    for batch in tqdm(batch_sizes):
        latent = torch.randn(batch, 512, device=device)
        img, _ = g([latent], truncation=truncation, truncation_latent=truncation_latent)
        feat = inception(img)[0].view(img.shape[0], -1)
        features.append(feat.to('cpu'))

    features = torch.cat(features, 0)

    return features


@torch.no_grad()
def extract_feature_from_generator_fn(generator_fn, inception, device='cuda', total=1000):
    features = []

    for batch in tqdm(generator_fn, total=total):
        try:
            feat = inception(batch)[0].view(batch.shape[0], -1)
            features.append(feat.to('cpu'))
        except:
            break
    features = torch.cat(features, 0).detach()
    return features.numpy()


def calc_fid(sample_features, real_features=None, real_mean=None, real_cov=None, eps=1e-6):
    sample_mean = np.mean(sample_features, 0)
    sample_cov = np.cov(sample_features, rowvar=False)

    if real_features is not None:
        real_mean = np.mean(real_features, 0)
        real_cov = np.cov(real_features, rowvar=False)

    cov_sqrt, _ = linalg.sqrtm(sample_cov @ real_cov, disp=False)

    if not np.isfinite(cov_sqrt).all():
        print('product of cov matrices is singular')
        offset = np.eye(sample_cov.shape[0]) * eps
        cov_sqrt = linalg.sqrtm((sample_cov + offset) @ (real_cov + offset))

    if np.iscomplexobj(cov_sqrt):
        if not np.allclose(np.diagonal(cov_sqrt).imag, 0, atol=1e-3):
            m = np.max(np.abs(cov_sqrt.imag))

            raise ValueError(f'Imaginary component {m}')

        cov_sqrt = cov_sqrt.real

    mean_diff = sample_mean - real_mean
    mean_norm = mean_diff @ mean_diff

    trace = np.trace(sample_cov) + np.trace(real_cov) - 2 * np.trace(cov_sqrt)

    fid = mean_norm + trace

    return fid

def real_image_loader(dataloader, n_batches=10):
    counter = 0
    while counter < n_batches:
        counter += 1
        rgb_img = next(dataloader)[0]
        if counter == 1:
            vutils.save_image(0.5*(rgb_img+1), 'tmp_real.jpg')  
        yield rgb_img.cuda()




@torch.no_grad()
def image_generator(dataset, net_ae, net_ig, BATCH_SIZE=8, n_batches=500):
    counter = 0
    dataloader = iter(DataLoader(dataset, BATCH_SIZE, sampler=InfiniteSamplerWrapper(dataset), num_workers=4, pin_memory=False))
    n_batches = min( n_batches, len(dataset)//BATCH_SIZE-1 )
    while counter < n_batches:
        counter += 1
        rgb_img, _, _, skt_img = next(dataloader)
        rgb_img = F.interpolate( rgb_img, size=512 ).cuda()
        skt_img = F.interpolate( skt_img, size=512 ).cuda()

        gimg_ae, style_feat = net_ae(skt_img, rgb_img)
        g_image = net_ig(gimg_ae, style_feat)
        if counter == 1:
            vutils.save_image(0.5*(g_image+1), 'tmp.jpg')        
        yield g_image


@torch.no_grad()
def image_generator_perm(dataset, net_ae, net_ig, BATCH_SIZE=8, n_batches=500):
    counter = 0
    dataloader = iter(DataLoader(dataset, BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False))
    n_batches = min( n_batches, len(dataset)//BATCH_SIZE-1 )
    while counter < n_batches:
        counter += 1
        rgb_img, _, _, skt_img = next(dataloader)
        rgb_img = F.interpolate( rgb_img, size=512 ).cuda()
        skt_img = F.interpolate( skt_img, size=512 ).cuda()

        perm = true_randperm(rgb_img.shape[0], device=rgb_img.device)

        gimg_ae, style_feat = net_ae(skt_img, rgb_img[perm])
        g_image = net_ig(gimg_ae, style_feat)
        if counter == 1:
            vutils.save_image(0.5*(g_image+1), 'tmp.jpg')        
        yield g_image

In [ ]:
from math import log
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.nn.functional import interpolate


from torch.utils.data import DataLoader
from torchvision import utils as vutils

import os
import random
from torchvision.transforms.transforms import RandomGrayscale
from tqdm import tqdm
import pickle
import numpy as np

def train_gan():
    
    real_features = None
    inception = load_patched_inception_v3().cuda()
    inception.eval()

    percept = PerceptualLoss(model='net-lin', net='vgg', use_gpu=True)

    saved_image_folder = saved_model_folder = None
    log_file_path = None
    if saved_image_folder is None:
        saved_image_folder, saved_model_folder = make_folders(SAVE_FOLDER, 'GAN_'+TRIAL_NAME)
        log_file_path = saved_image_folder+'/../gan_log.txt'
        log_file = open(log_file_path, 'w')
        log_file.close()

    dataset = PairedMultiDataset(data_root_colorful, data_root_sketch_1, data_root_sketch_2, data_root_sketch_3, im_size=IM_SIZE_GAN, rand_crop=True)
    print( 'the dataset contains %d images.'%len(dataset))
    dataloader = iter(DataLoader(dataset, BATCH_SIZE_GAN, sampler=InfiniteSamplerWrapper(dataset), num_workers=DATALOADER_WORKERS, pin_memory=True))

    dataset_rgb = ImageFolder('/content/My_drive/MyDrive/rajat-Inspiron-3576/SEM7/DEEP_LEARNING/Project/Files/images', trans_maker_augment(512))
    dataset_skt = ImageFolder('/content/My_drive/MyDrive/rajat-Inspiron-3576/SEM7/DEEP_LEARNING/Project/Files/sketch', trans_maker_augment(512))
    


    net_ae = AE(nfc=NFC, nbr_cls=NBR_CLS)

    PRETRAINED_AE_PATH = None
    PRETRAINED_AE_ITER = 120

    if PRETRAINED_AE_PATH is None:
        PRETRAINED_AE_PATH = "/content/My_drive/MyDrive/rajat-Inspiron-3576/SEM7/DEEP_LEARNING/Project/Files/saved/train_results/AE_trial-pr-art-10-13-6-40/models/250.pth" 
    else:
        PRETRAINED_AE_PATH = PRETRAINED_AE_PATH + '/models/%d.pth'%PRETRAINED_AE_ITER 
    
    net_ae.load_state_dicts(PRETRAINED_AE_PATH)
    net_ae.cuda()
    net_ae.eval()

    
    net_ig = RefineGenerator_face(nfc=NFC, im_size=IM_SIZE_GAN).cuda()
    net_id = Discriminator(nc=3).cuda() 
    if MULTI_GPU:
        net_ae = nn.DataParallel(net_ae)
        net_ig = nn.DataParallel(net_ig)
        net_id = nn.DataParallel(net_id)

    net_ig_ema = copy_G_params(net_ig)

    opt_ig = optim.Adam(net_ig.parameters(), lr=2e-4, betas=(0.5, 0.999))
    opt_id = optim.Adam(net_id.parameters(), lr=2e-4, betas=(0.5, 0.999))

    
    if GAN_CKECKPOINT is not None:
        ckpt = torch.load(GAN_CKECKPOINT)
        net_ig.load_state_dict(ckpt['ig'])
        net_id.load_state_dict(ckpt['id'])
        net_ig_ema = ckpt['ig_ema']
        opt_ig.load_state_dict(ckpt['opt_ig'])
        opt_id.load_state_dict(ckpt['opt_id'])

    
    losses_g_img = AverageMeter()
    losses_d_img = AverageMeter()
    losses_mse = AverageMeter()
    losses_rec_s = AverageMeter()

    losses_rec_ae = AverageMeter()

    fixed_skt = fixed_rgb = fixed_perm = None

    fid=[ [0,0] ]

    for epoch in range(EPOCH_GAN):   
        for iteration in tqdm(range(50)):
            rgb_img, skt_img_1, skt_img_2, skt_img_3 = next(dataloader)
            
            rgb_img = rgb_img.cuda()

            rd = random.randint(0, 3) 
            if rd == 0:
                skt_img = skt_img_1.cuda()
            elif rd == 1:
                skt_img = skt_img_2.cuda()
            else:
                skt_img = skt_img_3.cuda()

            if iteration==0:
                fixed_skt = skt_img_3[:8].clone().cuda()
                fixed_rgb = rgb_img[:8].clone()
                fixed_perm = true_randperm(fixed_rgb.shape[0], 'cuda')

            
            gimg_ae, style_feats = net_ae(skt_img, rgb_img)
            g_image = net_ig(gimg_ae, style_feats)

            pred_r = net_id( rgb_img )        
            pred_f = net_id( g_image.detach() )   

            loss_d = d_hinge_loss(pred_r, pred_f) 
            
            net_id.zero_grad()
            loss_d.backward()
            opt_id.step()
            
            loss_rec_ae = F.mse_loss(gimg_ae, rgb_img) + F.l1_loss(gimg_ae, rgb_img)
            losses_rec_ae.update(loss_rec_ae.item(), BATCH_SIZE_GAN)
                
            
            pred_g = net_id(g_image)   
            loss_g = g_hinge_loss(pred_g) 

            if DATA_NAME == 'shoe':
                loss_mse = 10 * (F.l1_loss(g_image, rgb_img) + F.mse_loss(g_image, rgb_img))
            else:
                loss_mse = 10*percept( F.adaptive_avg_pool2d(g_image, output_size=256), F.adaptive_avg_pool2d(rgb_img, output_size=256)).sum()                
            losses_mse.update(loss_mse.item()/BATCH_SIZE_GAN, BATCH_SIZE_GAN)


            loss_all = loss_g + loss_mse

            if DATA_NAME == 'shoe':
                
                perm = true_randperm(BATCH_SIZE_GAN)
                img_ae_perm, style_feats_perm = net_ae(skt_img, rgb_img[perm])

                gimg_grey = net_ig(img_ae_perm, style_feats_perm)
                gimg_grey = gimg_grey.mean(dim=1, keepdim=True)
                real_grey = rgb_img.mean(dim=1, keepdim=True)
                loss_rec_grey = F.mse_loss( gimg_grey , real_grey )
                loss_all += 10 * loss_rec_grey 

            net_ig.zero_grad()
            loss_all.backward()
            opt_ig.step()

            for p, avg_p in zip(net_ig.parameters(), net_ig_ema):
                avg_p.mul_(0.999).add_( p.data, alpha=0.001)

            
            losses_g_img.update(pred_g.mean().item(), BATCH_SIZE_GAN)
            losses_d_img.update(pred_r.mean().item(), BATCH_SIZE_GAN)
            


            if iteration % SAVE_IMAGE_INTERVAL == 0:
                with torch.no_grad():
                
                    backup_para_g = copy_G_params(net_ig)
                    load_params(net_ig, net_ig_ema)

                    gimg_ae, style_feats = net_ae(fixed_skt, fixed_rgb)
                    gmatch = net_ig(gimg_ae, style_feats)
                    
                    

                    gimg_ae_perm, style_feats = net_ae(fixed_skt, fixed_rgb[fixed_perm])
                    gmismatch = net_ig(gimg_ae_perm, style_feats)

                    gimg = torch.cat([ F.interpolate(fixed_rgb, IM_SIZE_GAN), 
                                       F.interpolate(fixed_skt.repeat(1,3,1,1), IM_SIZE_GAN), 
                                       gmatch, 
                                       F.interpolate(gimg_ae, IM_SIZE_GAN),
                                       gmismatch, 
                                       F.interpolate(gimg_ae_perm, IM_SIZE_GAN)])
                    
                    vutils.save_image(gimg, f'{saved_image_folder}/img_iter_{epoch}_{iteration}.jpg', normalize=True, range=(-1, 1))
                    del gimg
                    
                    load_params(net_ig, backup_para_g)

            if iteration % LOG_INTERVAL == 0:
                log_msg = 'Iter: [{0}/{1}] G: {losses_g_img.avg:.4f}  D: {losses_d_img.avg:.4f}  MSE: {losses_mse.avg:.4f}  Rec: {losses_rec_s.avg:.5f}  FID: {fid:.4f}'.format(epoch, 
                            iteration, losses_g_img=losses_g_img, losses_d_img=losses_d_img, losses_mse=losses_mse, losses_rec_s=losses_rec_s, fid=fid[-1][0])

                print(log_msg)
                print('%.5f'%(losses_rec_ae.avg))

                if log_file_path is not None:
                    log_file = open(log_file_path, 'a')
                    log_file.write(log_msg+'\n')
                    log_file.close()
                    
                losses_g_img.reset()
                losses_d_img.reset()
                losses_mse.reset()
                losses_rec_s.reset()
                losses_rec_ae.reset()

            if iteration % SAVE_MODEL_INTERVAL == 0 or iteration+1 == 50:
                print('Saving history model')
                torch.save( {'ig': net_ig.state_dict(),
                            'id': net_id.state_dict(),
                            'ae':net_ae.state_dict(),
                            'ig_ema': net_ig_ema,
                            'opt_ig': opt_ig.state_dict(), 
                            'opt_id': opt_id.state_dict(),
                            }, '%s/%d.pth'%(saved_model_folder, epoch))
        
            if iteration % FID_INTERVAL == 0 and iteration>1:
                print("calculating FID ...")
                fid_batch_images = FID_BATCH_NBR
                if real_features is None:
                    if os.path.exists('%s_fid_feats.npy'%(DATA_NAME)):
                        real_features = pickle.load(open('%s_fid_feats.npy'%(DATA_NAME), 'rb'))
                    else:
                        real_features = extract_feature_from_generator_fn( 
                                            real_image_loader(dataloader, n_batches=fid_batch_images), inception )
                        real_mean = np.mean(real_features, 0)
                        real_cov = np.cov(real_features, rowvar=False)
                        pickle.dump({'feats': real_features, 'mean': real_mean, 'cov': real_cov}, open('%s_fid_feats.npy'%(DATA_NAME),'wb') ) 
                        real_features = pickle.load(open('%s_fid_feats.npy'%(DATA_NAME), 'rb'))
                
                sample_features = extract_feature_from_generator_fn( image_generator(dataset, net_ae, net_ig, n_batches=fid_batch_images), inception, total=fid_batch_images )
                cur_fid = calc_fid(sample_features, real_mean=real_features['mean'], real_cov=real_features['cov'])
                sample_features_perm = extract_feature_from_generator_fn( image_generator_perm(dataset, net_ae, net_ig, n_batches=fid_batch_images), inception, total=fid_batch_images )
                cur_fid_perm = calc_fid(sample_features_perm, real_mean=real_features['mean'], real_cov=real_features['cov'])
                
                fid.append( [cur_fid, cur_fid_perm] )
                print('fid:', fid)
                if log_file_path is not None:
                    log_file = open(log_file_path, 'a')
                    log_msg = 'fid: %.5f, %.5f'%(fid[-1][0], fid[-1][1])
                    log_file.write(log_msg+'\n')
                    log_file.close()


In [ ]:
## training GAN

In [ ]:
train_gan()

/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:82: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth


  0%|          | 0.00/91.2M [00:00<?, ?B/s]

Setting up Perceptual loss...


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

...


100%|██████████| 10/10 [00:11<00:00,  1.15s/it]

 82%|████████▏ | 41/50 [05:10<02:56, 19.57s/it]

fid: [[0, 0], [164.52637678381708, 164.2761330810639], [160.66537227129885, 161.45784831307304], [155.8688076388467, 156.86813830105928], [145.21074956213764, 149.12100274863133], [147.84916982085167, 146.3539444249989], [160.18945710357093, 168.5178432737312], [154.1534043331386, 153.92320951213122], [170.55732041292666, 166.3139598736498], [159.39470684928705, 163.0703656213507], [152.7981090162765, 151.96775313666058], [153.8181465593576, 158.13300023783137], [145.6208782933127, 145.06669541400416], [137.69543695932273, 147.6602819895292], [151.73137696188394, 155.92675791825747], [137.5612832669261, 144.4680563617743], [148.1701648284955, 152.38076218677966], [132.3254167777917, 138.7321935129155], [136.4933488901036, 142.0188752754113], [142.34314373403996, 146.1609604754796], [138.55198227617737, 145.21649940664705], [145.60165842204074, 145.94027671510378], [138.44016706171075, 134.47773302226904], [138.13897868857768, 141.52151432253646], [133.9982049771424, 140.10537620991067]

 98%|█████████▊| 49/50 [05:26<00:02,  2.98s/it]

Saving history model


100%|██████████| 50/50 [05:29<00:00,  6.59s/it]
